In [4]:
from PIL import Image
from transformers import ViTImageProcessor, ViTModel, CLIPImageProcessor, CLIPModel

c:\Users\detac\AppData\Local\Programs\Python\Python39\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
import torch
import torch.nn as nn

class Hypernetwork(nn.Module):
    def __init__(self, input_size, num_layers, layer_size, chunk_size, chunk_emb_size, num_chunks):
        super().__init__()
        input_size = input_size + chunk_emb_size
        self.chunk_embeddings = self._generate_chunk_embeddings(chunk_emb_size, num_chunks) 

        hypernet_layers = self._prepare_layers(num_layers, layer_size, input_size, chunk_size)
        self.hypernet = nn.Sequential(*hypernet_layers)

    def forward(self, x):
        fast_weights = []
        for chunk_emb in self.chunk_embeddings:
            cat_ = torch.cat((x, chunk_emb), dim=1)
            fast_weight_chunk = self.hypernet(cat_)
            fast_weights.append(fast_weight_chunk)
        return fast_weights

    def _prepare_layers(self, num_layers, layer_size, input_size, chunk_size):
        input_layer = nn.Linear(in_features=input_size, out_features=layer_size)
        layers = [input_layer, nn.ReLU()]
        for _ in range(num_layers-1):
            layer = nn.Linear(layer_size, layer_size)
            layers.append(layer)
            layers.append(nn.ReLU())

        layers.append(nn.Linear(in_features=layer_size, out_features=chunk_size))
        layers.append(nn.Sigmoid())
        return layers
        
    def _generate_chunk_embeddings(self, chunk_emb_size, num_chunks):
        return [torch.rand((1, chunk_emb_size)) for _ in range(num_chunks)]

In [11]:
import torch.nn.functional as F

class Linear_fw(nn.Linear): #used in MAML to forward input with fast weight
    def __init__(self, in_features, out_features):
        super().__init__(in_features, out_features)
        self.weight.fast = self.weight
        self.bias.fast = self.bias

    def forward(self, x):
        if self.weight.fast is not None and self.bias.fast is not None:
            out = F.linear(x, self.weight.fast, self.bias.fast) #weight.fast (fast weight) is the temporaily adapted weight
        else:
            out = F.linear(x, self.weight, self.bias)
        return out

In [12]:
class MLP_FW(nn.Module):
    def __init__(self, input_size, num_layers, layer_size, num_classes):
        super().__init__()
        layers = self._generate_layers(input_size, num_layers, layer_size, num_classes)
        
        self.net = nn.Sequential(*layers)

    def _generate_layers(self, input_size, num_layers, layer_size, num_classes):
        layers = [Linear_fw(input_size, layer_size), nn.ReLU()]
        for _ in range(num_layers-2):
            layers.append(Linear_fw(layer_size, layer_size))
            layers.append(nn.ReLU())

        layers.append(Linear_fw(layer_size, num_classes))
        layers.append(nn.ReLU())

        return layers

In [24]:
mlp = MLP_FW(768, 4, 128, 5)

In [25]:
sum(p.numel() for p in mlp.parameters())

132101

In [13]:
input_embedding = torch.rand((1,768))

In [14]:
hypernet = Hypernetwork(768, 4, 500, 128, 20, 2)

In [15]:
updates = hypernet(input_embedding)

In [2]:
from transformers import ViTForImageClassification

In [3]:
vit_image_processor = ViTImageProcessor.from_pretrained("google/vit-base-patch16-224")
clip_image_processor = CLIPImageProcessor.from_pretrained("openai/clip-vit-base-patch32")

img = Image.open("/home/lukasz/binary-hyper-maml/filelists/emnist/emnist/13/11.png").convert('RGB')

In [4]:
img

In [5]:
img.size

(28, 28)

In [6]:
inputs = vit_image_processor(images=img, return_tensors="pt")
pixel_values = inputs.pixel_values

In [7]:
model = ViTForImageClassification.from_pretrained('google/vit-base-patch16-224')

In [8]:
outptut = model(pixel_values)

In [9]:
outptut[0].shape

torch.Size([1, 1000])

In [10]:
feature_extractor = ViTModel.from_pretrained('google/vit-base-patch16-224')

Some weights of ViTModel were not initialized from the model checkpoint at google/vit-base-patch16-224 and are newly initialized: ['vit.pooler.dense.weight', 'vit.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [11]:
outputs = feature_extractor(pixel_values)
sequence_output = outputs[0]

sequence_output[:, 0, :].shape

torch.Size([1, 768])